In [26]:
from typing_extensions import dataclass_transform
import psycopg2
import datetime
from matplotlib import pyplot as plt
HOST = "localhost"
USER = "postgres"
PASSWORD = "postgres"
DATABASE = "postgres"
# Open connection
conn = psycopg2.connect("host=%s dbname=%s user=%s password=%s" % (HOST, DATABASE, USER, PASSWORD))
# Open a cursor to send SQL commands
cur = conn.cursor()



In [28]:
## AFFCIHAGE DE LA PROPORTION DE CONSULTATIONS PASSANT PAR LA PAGE SUIVI TRAJECTOIRE
def nb_consultations_traj():
    sql= "SELECT COUNT(*) FROM consultations.analyses_territoriales WHERE page='suivi_trajectoire'"
    
    cur.execute(sql)
    
    # Fetch data line by line
    raw = cur.fetchone()
    return raw[0]

print(nb_consultations_traj)

<function nb_consultations_traj at 0x7f69429d5790>


In [32]:
from typing_extensions import dataclass_transform
import psycopg2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

from irrégularités import correction_themes, correction_noms

HOST = "localhost"
USER = "postgres"
PASSWORD = "postgres"
DATABASE = "postgres"
# Open connection
conn = psycopg2.connect("host=%s dbname=%s user=%s password=%s" % (HOST, DATABASE, USER, PASSWORD))
# Open a cursor to send SQL commands
cur = conn.cursor()

sql = """SELECT consultations.consultations_indicateurs.id_indicateur,
meta.indicateur.nom,
meta.indicateur.ui_theme,
consultations.consultations_indicateurs.date,
consultations.consultations_indicateurs.region,
consultations.consultations_indicateurs.provenance
FROM consultations.consultations_indicateurs
LEFT JOIN meta.indicateur ON consultations.consultations_indicateurs.id_indicateur = meta.indicateur.id;"""
cur.execute(sql)
raw = cur.fetchall()


sql1="SELECT * FROM consultations.consultations_indicateurs WHERE consultations.consultations_indicateurs.provenance NOT LIKE 'tableaux_de_bord'"
cur.execute(sql1)
raw1 = cur.fetchall()

conn.close()

df = pd.DataFrame(raw, columns=["id_indicateur", "nom", "ui_theme", "date", "region", "provenance"])

df = correction_themes(df)
df = correction_noms(df)


In [31]:
## si on veut exclure la provenance tableaux de bord car elle biaise les proportions
liste=[]
for i in range(df.shape[0]):
    if df.provenance[i]!="tableaux_de_bord":
        liste.append(i)


df_new=df.drop(liste)

print(df.shape)
print(df_new.shape)




(20622, 6)
(698, 6)
